# LLM Agent for event listings
In this notebook, we set up a simple workflow for an agent to suggest some cool events from the [Boston Calendar](https://www.thebostoncalendar.com/).  This can be run locally or on Colab, but requires you to have access to [OpenAI's API](https://openai.com/blog/openai-api). 

Note - use of the API is available for free trial, but is paid after that.

In [14]:
from langchain_openai import ChatOpenAI
from langchain.agents import (
    tool, 
    create_react_agent,
    load_tools,
    AgentExecutor
)
from langchain.agents import load_tools
from langchain_core.prompts import PromptTemplate
from datetime import datetime, timedelta, date
from langchain_core.messages.system import SystemMessage
from langchain_core.messages.human import HumanMessage

# this is for simple scraping tool
import requests
from random import sample
from bs4 import BeautifulSoup

In [2]:
# using .env file for GPT API key
from dotenv import load_dotenv
load_dotenv()
llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(model=llm_model)

### Setting up the Reasoning + Act (ReAct) prompt
To make the output of the model follow a particular format, we need to provide it with that format.  This enables us to use a parser to understand what tools and parameters the model would suggest using.

In [3]:
# prompt is adapted from LangChain's example: https://api.python.langchain.com/en/latest/agents/langchain.agents.react.agent.create_react_agent.html
# action input is required to be a string - otherwise I found it would use function calls as inputs
template = '''Answer the following questions as best you can. \
You will need to break your response into steps, each which may use a different tool. \
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: an input to the action, usually an empty string
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question. 

Think step by step! Begin!

Question: {input}
Thought:{agent_scratchpad}'''

prompt = PromptTemplate.from_template(template)

#### Simple workflow
Here we set up an agent that just has access to a function to tell today's date.  We'll be making the workflow "verbose" so that we can see each step the process takes.  You'll see in green the text produced by the agent and in blue the output from the function.  The final result is provided as a json with both input and output.

In [5]:
# notice the extensive, detailed docstrings - this is for the LLM
@tool
def today(text: str) -> str:
    """Returns today's date, use this when you need to get today's date.
    The input should always be an empty string."""
    return str(date.today())

In [6]:
tools = [today]
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, 
                               verbose=True, 
                               handle_parsing_errors=True,
                              max_iterations=5)
agent_executor.invoke({"input": "What day is it today?"})



> Entering new AgentExecutor chain...
I need to find out today's date.
Action: today
Action Input: 2024-02-25I now know the final answer
Final Answer: Today is February 25, 2024.

> Finished chain.


{'input': 'What day is it today?', 'output': 'Today is February 25, 2024.'}

#### Local events workflow
These next functions provide two new tools for our agent; one that provides the current "weekend number" in the month and one that scrapes Boston Calendar for event information.  Most of this is just scraping and formatting Boston Calendar, but the functions with the tool decorators will be provided to the model.

Then, we set up our ReAct agent as before.

In [7]:

def parse_for_calendar(text: str) -> str:
    # utility for converting text input to boston calendar URL
    if len(text) == 1:
        url = f'https://www.thebostoncalendar.com/events?day=10&month=2&weekend={text}&year=2024'
    # this will fail if the string provided is not a date
    else:
        try:
            day_of_month = datetime.strptime(text, '%Y-%M-%d').day    
            url = f'https://www.thebostoncalendar.com/events?day={day_of_month}&month=2&year=2024'
        except ValueError:
            return 
    return url

@tool
def weekend(text: str) -> str:
    """Returns the single-digit weekend number for this weekend, \
    use this for any questions related to the weekend date. \
    The input should always be an empty string, \
    and this function will always return the weekend number."""
    today = datetime.now()
    
    # Calculate the weekend number within the month
    first_day_of_month = today.replace(day=1)
    weekend_number_within_month = (today - first_day_of_month).days // 7 + 1

    return weekend_number_within_month

@tool
def get_events(text: str) -> str:
    """Returns local events. \ 
    The input is either a date string in the format YYYY-MM-DD, \
    or it is a single-digit weekend number.\
    The input cannot be a function call.\
    This function will return a list where \
    each element contains an event name, date and location as a tuple.\
    This function should be used to provide complete information about events."""
    # use the parsing utility to get a formatted url
    url = parse_for_calendar(text)
    if url is None:
        # give the LLM a useful response
        return f'Input "{text}" is not in the right format - it needs to be a date string or a weekend number'
    response = requests.get(url)     
    
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract data
    events = soup.find_all('div', class_='info')

    all_events = []
    for event in events:
        title = event.find('h3').text.strip()
        date = event.find('p', class_='time').text.strip()
        location = event.find('p', class_='location').text.strip()
        all_events.append((title, date, location))

    # randomly select a few, provide as list
    choices = sample(all_events, 3)
    
    return choices


In [8]:
tools = [today, weekend, get_events]
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, 
                               verbose=True, 
                               handle_parsing_errors=True,
                              max_iterations=5)
agent_executor.invoke({"input": "What is going on this weekend?"})



> Entering new AgentExecutor chain...
I need to find out the weekend number first before getting the events happening this weekend.
Action: weekend
Action Input: 4Now that I have the weekend number, I can use it to get the events happening this weekend.
Action: get_events
Action Input: 4[('Boston Public Market Winter Movie Series', 'Sunday, Feb 25, 2024 1:00p', 'Boston Public Market'), ('Winter Wonderbands: Live Music Sundays', 'Sunday, Feb 25, 2024 1:00p', 'Harpoon Brewery'), ('BYO! Underground Comedy', 'Saturday, Feb 24, 2024 7:00p', 'The Secret Loft')]I now know the events happening this weekend.
Final Answer: The events happening this weekend are the Boston Public Market Winter Movie Series, Winter Wonderbands: Live Music Sundays, and BYO! Underground Comedy.

> Finished chain.


{'input': 'What is going on this weekend?',
 'output': 'The events happening this weekend are the Boston Public Market Winter Movie Series, Winter Wonderbands: Live Music Sundays, and BYO! Underground Comedy.'}

We can see here that the output provides three random examples of events coming up this weekend.  One note - if you run this during the weekend, it will return you the current weekend.

Our "today" tool also enables the agent to get today's events.

In [10]:
tools = [today, weekend, get_events]
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, 
                               verbose=True, 
                               handle_parsing_errors=True,
                              max_iterations=5)
agent_executor.invoke({"input": "What is going on today?"})



> Entering new AgentExecutor chain...
I need to find out today's date to answer this question.
Action: today
Action Input: ""2024-02-25Now that I know today's date, I can proceed to find out what events are happening today.
Action: get_events
Action Input: 2024-02-25[('Boqueria: Spanish-style tapas & brunch | Seaport', 'Sunday, Feb 25, 2024 goes until 03/31', 'Boqueria'), ('Commonwealth Museum', 'Sunday, Feb 25, 2024 goes until 03/31', 'Commonwealth Museum'), ('Wolf Vostell: Dé-coll/age Is Your Life', 'Sunday, Feb 25, 2024 goes until 05/05', 'Harvard Art Museums')]I now know the events happening today. 
Final Answer: The events happening today are Boqueria: Spanish-style tapas & brunch at Seaport, Commonwealth Museum, and Wolf Vostell: Dé-coll/age Is Your Life at Harvard Art Museums.

> Finished chain.


{'input': 'What is going on today?',
 'output': 'The events happening today are Boqueria: Spanish-style tapas & brunch at Seaport, Commonwealth Museum, and Wolf Vostell: Dé-coll/age Is Your Life at Harvard Art Museums.'}

And we can even ask for specific dates!

In [11]:
agent_executor.invoke({"input": "What is going on 2/28/24?"})



> Entering new AgentExecutor chain...
I need to find out what events are happening on February 28, 2024.
Action: get_events
Action Input: 2024-02-28[('The Best Works of Outdoor Public Art to See Around Boston', 'Wednesday, Feb 28, 2024 goes until 12/31', 'Boston'), ('Toro Boston: Dynamic Spanish Tapas | South End', 'Wednesday, Feb 28, 2024 goes until 03/31', 'Toro Boston'), ('Rooftop at Legal Sea Foods', 'Wednesday, Feb 28, 2024 goes until 03/31', 'Legal Sea Foods')]There are several events happening on February 28, 2024 in Boston.
Final Answer: On February 28, 2024, you can attend "The Best Works of Outdoor Public Art to See Around Boston" in Boston, "Toro Boston: Dynamic Spanish Tapas | South End" at Toro Boston, and "Rooftop at Legal Sea Foods" at Legal Sea Foods.

> Finished chain.


{'input': 'What is going on 2/28/24?',
 'output': 'On February 28, 2024, you can attend "The Best Works of Outdoor Public Art to See Around Boston" in Boston, "Toro Boston: Dynamic Spanish Tapas | South End" at Toro Boston, and "Rooftop at Legal Sea Foods" at Legal Sea Foods.'}

#### More fun experiments
I wanted to see what happened if I passed something output from the workflow above to our "friendly" prompt we experimented with in the post on [fine tuning](https://bpben.github.io/friend_ft_4/).  It actually does pretty well, but it strips away some of the useful information that is provided by the default style.

In [12]:
system_prompt = """Your name is Friend.  You are having a conversation with your close friend Ben. 
You and Ben are sarcastic and poke fun at one another. 
But you care about each other and support one another."""

input_prompt = """You know the following information:
The events happening tomorrow are: 
1. Pammy’s: Italian Neighborhood Trattoria - Monday, Feb 05, 2024 goes until 12/31 at Pammy's
2. Stoneham Town Common Skating Rink - Monday, Feb 05, 2024 goes until 03/24 at Stoneham Town Common Rink
3. Park-9 Dog Bar: Indoor Dog Park & Bar | Everett - Monday, Feb 05, 2024 goes until 03/31 at Park-9 Dog Bar

Ben: What's going on tomorrow?"""

In [15]:
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=input_prompt),
]

llm.invoke(messages)

AIMessage(content="Friend: Oh, just the usual, Ben. I'm sure you'll be torn between ice skating with the kids or taking Fido out for a drink at the dog bar. Decisions, decisions!")